# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("../output_data/cities.csv", index_col=0)
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Doha,25.2867,51.5333,73.40,43,0,9.22,QA,1610196904
1,Mataura,-46.1927,168.8643,54.00,84,56,5.01,NZ,1610196904
2,Saskylakh,71.9167,114.0833,-19.98,86,100,12.21,RU,1610196905
3,Port Elizabeth,-33.9180,25.5701,84.20,61,20,12.66,ZA,1610196821
4,Talara,-4.5772,-81.2719,69.80,88,75,14.97,PE,1610196681


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
# Store latitude and longitude in locations
locations = cities_df[["Lat", "Lng"]].astype(float)
# Fill NaN values and convert to float
humidity = cities_df["Humidity"].astype(float)

In [5]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=00, point_radius=5)
# Add layer
fig.add_layer(heat_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Cities with a max temperature lower than 80 degrees but higher than 70,
# Wind speed less than 10 mph and zero cloudiness. 
hotel_df = cities_df.loc[(cities_df["Max Temp"] < 80) & (cities_df["Max Temp"] > 76) 
                              & (cities_df["Wind Speed"] < 10) & (cities_df["Cloudiness"] == 0), :]

hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
20,Darnah,32.7670,22.6367,78.06,39,0,6.49,LY,1610196913
75,Erechim,-27.6342,-52.2739,77.00,69,0,9.22,BR,1610196939
229,Venado Tuerto,-33.7456,-61.9688,77.00,38,0,5.01,AR,1610197011
282,Tapejara,-28.0681,-52.0139,76.28,73,0,7.40,BR,1610197038
341,Verāval,20.9000,70.3667,78.93,54,0,8.55,IN,1610197066
463,Swan Hill,-35.3378,143.5544,78.66,27,0,6.11,AU,1610197126
471,Saint-Louis,16.3333,-15.0000,78.80,47,0,4.61,SN,1610197130


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel = hotel_df[["City", "Country", "Lat", "Lng"]].copy()
hotel["Hotel Name"]=""
hotel

,City,Country,Lat,Lng,Hotel Name
20,Darnah,LY,32.7670,22.6367,
75,Erechim,BR,-27.6342,-52.2739,
229,Venado Tuerto,AR,-33.7456,-61.9688,
282,Tapejara,BR,-28.0681,-52.0139,
341,Verāval,IN,20.9000,70.3667,
463,Swan Hill,AU,-35.3378,143.5544,
471,Saint-Louis,SN,16.3333,-15.0000,


In [8]:
params = {
    "radius":5000, 
    "types":"lodging",
    "key":g_key    
}
for index, row in hotel.iterrows():
    lat=row["Lat"]
    lng=row["Lng"]
    params["location"]=f"{lat},{lng}"
    base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    name_address=requests.get(base_url, params=params)
    name_address=name_address.json()
    try:
        hotel.loc[index,"Hotel Name"]=name_address["results"][0]["name"]        
    except(KeyError, IndexError):
        print(f"Missing a hotel name for {row['City']}")
hotel

Missing a hotel name for Saint-Louis


,City,Country,Lat,Lng,Hotel Name
20,Darnah,LY,32.7670,22.6367,فندق الفردوس
75,Erechim,BR,-27.6342,-52.2739,Hotel Buongiorno
229,Venado Tuerto,AR,-33.7456,-61.9688,La Bianca.
282,Tapejara,BR,-28.0681,-52.0139,Hotel Fontana
341,Verāval,IN,20.9000,70.3667,Lords Inn Somnath
463,Swan Hill,AU,-35.3378,143.5544,Lazy River Motor Inn
471,Saint-Louis,SN,16.3333,-15.0000,


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel.iterrows()]
location = hotel[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(location, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))